## Ensemble Learning
- Getting preprocessed correlated data from Google Drive
- Using KNN, SVM, Logistic Regression and Ridge Regression for base modesl
- Using Bagging

## Import and Setting

In [2]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import shutil
import os
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler

Mounted at /content/drive



# Adding Korean

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import unicodedata

In [4]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 123586 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Setting up fo

In [5]:
fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 8, 'font.family': 'NanumGothic'})
mpl.rcParams['axes.unicode_minus'] = False

## Getting Preprocessed Data


In [86]:
# Getting CSV
directory_path = '/content/drive/MyDrive/CSVs/JejuWeather'
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

# Getting paths to obs directory
obs_dirs_paths = []
for obs in csv_files:
  obs_korean = obs.replace('.csv', '')
  obs_dirs_paths.append(os.path.join(directory_path, obs_korean))

# Getting paths to relavance csv of each obs directory
related_columns_csvs = []
for obs in obs_dirs_paths:
  temp = []
  temp.append(obs)
  temp.append(os.path.join(obs,[f for f in os.listdir(obs) if f.startswith('relevance_')][0]))
  related_columns_csvs.append(temp)

## Common function & Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [87]:
def three_days(obs_csv_relevance):
  data = pd.read_csv(obs_csv_relevance).sort_values('baseDate')
  prev_day_1 = data.shift(1).drop(columns=['baseDate'])
  prev_day_2 = data.shift(2).drop(columns=['baseDate'])
  prev_day_3 = data.shift(3).drop(columns=['baseDate'])


  combined_data = pd.concat([data['baseDate'], data['dailyRainfall'],prev_day_1.add_suffix('_prev1'), prev_day_2.add_suffix('_prev2'), prev_day_3.add_suffix('_prev3')], axis=1)
  combined_data = combined_data.dropna()
  combined_data = combined_data.reset_index(drop=True)

  return combined_data

In [88]:
def x_y_split(obs_csv_relevance):
  data = three_days(obs_csv_relevance)

  # X: related features
  X_columns = data.drop(columns=['baseDate', 'dailyRainfall'])

  # y: target
  y = (data['dailyRainfall'] > 0).astype(int)
  y = y.reset_index(drop=True)

  return X_columns, y

## Base Models: Linear Regression

In [89]:
from sklearn.linear_model import LogisticRegression

In [90]:
def jeju_weather_linear(obs_csv_relevance):
  # Splitting
  X, y = x_y_split(obs_csv_relevance)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train_scaled = StandardScaler().fit_transform(X_train)
  X_test_scaled = StandardScaler().fit_transform(X_test)
  # Model & Training
  logistic_reg = LogisticRegression(max_iter=1000)
  logistic_reg.fit(X_train_scaled, y_train)

  # Prediction
  accuracy = logistic_reg.score(X_test_scaled, y_test)
  print("Lienar Regression Accuracy:", accuracy)
  return logistic_reg

## KNN

In [101]:
from sklearn.neighbors import KNeighborsClassifier

In [102]:
def jeju_weather_knn(obs_csv_relevance):
  # Splitting
  X, y = x_y_split(obs_csv_relevance)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train_scaled = StandardScaler().fit_transform(X_train)
  X_test_scaled = StandardScaler().fit_transform(X_test)

  knn = KNeighborsClassifier(n_neighbors=5)
  knn.fit(X_train_scaled, y_train)

  accuracy = knn.score(X_test_scaled, y_test)
  print("KNN Accuracy:", accuracy)
  return knn

## Support Vector Machine

## KMeans Cluster: Exclude

In [91]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [92]:
def find_elbow_point(wcss):
    deltas = np.diff(wcss)
    second_deltas = np.diff(deltas)
    elbow_point = np.argwhere(second_deltas > 0)[0][0] + 2
    return elbow_point

In [93]:
def jeju_weather_kmeans(obs_csv_relevance):
  # Features and scaling
  three_days_combined = three_days(obs_csv_relevance)
  features = three_days_combined.drop(columns=['baseDate', 'dailyRainfall'])
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(features)

  # getting optimal nums of cluters
  wcss = []
  max_clusters = 10
  for i in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, n_init=10, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

  optimal_k = find_elbow_point(wcss)
  # Model & Training
  kmeans = KMeans(n_clusters=optimal_k, n_init=10, random_state=42)
  three_days_combined['cluster'] = kmeans.fit_predict(scaled_features)
  print(three_days_combined['cluster'].value_counts())


## Training

In [104]:
for related_columns_csv in related_columns_csvs:
  obs_dir_path = related_columns_csv[0]
  print(obs_dir_path.split("/")[-1])
  current_log_reg_model = jeju_weather_linear(related_columns_csv[1])
  current_log_reg_model = jeju_weather_knn(related_columns_csv[1])
  if os.path.exists(os.path.join(obs_dir_path, 'linear_reg_model.pkl')):
    os.remove(os.path.join(obs_dir_path, 'linear_reg_model.pkl'))
  joblib.dump(current_log_reg_model, os.path.join(obs_dir_path, 'linear_reg_model.pkl'))

  break

마라도
Lienar Regression Accuracy: 0.6897001303780965
KNN Accuracy: 0.6649282920469362
